In [4]:
import dspy
import random
from typing import List, Dict, Any
from dataclasses import dataclass

# Your original corpus passages
MY_CORPUS_PASSAGES = [
    "1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) ",
    "2) Full Expression - it takes a lot of effort for one to understand who they are when they are comfortable and how to channel the most real expressions of themselves what holds people back? Shyness, distraction (inability to focus on that which they want express)",
    "3) Expect Rising - this means our expectations are constantly rising. Kind of in line with \"give em an inch they'll take a mile\" also related to law of diminishing returns -\"The law of diminishing returns states that in all productive processes, adding more of one factor of production, while holding all others constant (\"ceteris paribus\"), will at some point yield lower incremental per-unit returns.\" Tooo much ice cream too much cash",
    "4) The Power of Pettiness - is the idea that pettiness is the destroyer of all people. That even the best of us can't truly be unaffected by the petty bullshit around us. They can however minimize its effects. In my own life I am obliterated by friends and coworker's snide remarks and judgments. How do I minimize its effects? By talking myself down, deep breaths, weed, alcohol, revenge...(this is the end of the first four points that were the strongest when I first started making these ideas on a small scrap of paper from the sides on union film sets.)",
    "5) Various meditation - I am a big fan of meditation techniques that help train the mind from moment to moment. There are various techniques that help provide concentration and awareness in the passing moments within shorter intervals than traditional formal meditation. While using a timer to make sure you meditate for at least 10 minutes is helpful. A stopwatch to monitor your awareness of time and to also use time to monitor your concentration and awareness. A and C are essential to have Full Expression. Update 2-26-19 - Passive observation ala Krishnamurti. Just chill. Dont do anything. Just observe whatever is present. Don't react to thoughts just observe them. Don't interact with them. If they are mind blowing write them down. Then return to just chilling and observing."
]

class IdeaConsistencyJudge(dspy.Signature):
    """Judge if the answer is consistent with the personal philosophical ideas and concepts in the context."""
    
    context = dspy.InputField(desc="Philosophical context from personal ideas corpus")
    question = dspy.InputField(desc="Question about life philosophy or personal growth")
    answer = dspy.InputField(desc="Answer to evaluate for consistency")
    consistent_with_ideas: bool = dspy.OutputField(desc="Is the answer consistent with the philosophical ideas in the context?")
    explanation = dspy.OutputField(desc="Detailed reasoning for why the answer is or isn't consistent with the ideas")

# Create the judge with chain of thought reasoning
judge = dspy.ChainOfThought(IdeaConsistencyJudge)

def idea_consistency_metric(example, pred):
    """
    Evaluate how well an answer aligns with the personal philosophy corpus.
    Returns both the boolean consistency score and the explanation.
    """
    judgment = judge(
        context=example.context, 
        question=example.question, 
        answer=pred.answer
    )
    
    return {
        'consistent': judgment.consistent_with_ideas,
        'explanation': judgment.explanation,
        'score': 1.0 if judgment.consistent_with_ideas else 0.0
    }

class MyIdeasRAGProgram(dspy.Module):
    """RAG program that retrieves from personal ideas corpus and includes context in predictions."""
    
    def __init__(self, corpus_passages: List[str]):
        super().__init__()
        self.corpus = corpus_passages
        self.generator = dspy.ChainOfThought("question -> answer")
        
    def retrieve_relevant_ideas(self, question: str, top_k: int = 2) -> List[str]:
        """
        Simple retrieval based on keyword matching.
        In practice, you'd use your ideas1.faiss index here.
        """
        # Simple keyword-based scoring for demo
        keywords = question.lower().split()
        scored_passages = []
        
        for passage in self.corpus:
            score = sum(1 for keyword in keywords if keyword in passage.lower())
            scored_passages.append((score, passage))
        
        # Sort by score and return top_k
        scored_passages.sort(key=lambda x: x[0], reverse=True)
        return [passage for _, passage in scored_passages[:top_k]]
    
    def forward(self, question: str):
        # Retrieve relevant context from ideas corpus
        relevant_ideas = self.retrieve_relevant_ideas(question)
        context = "\n\n".join(relevant_ideas)
        
        # Generate answer using the context
        answer = self.generator(question=f"Context: {context}\n\nQuestion: {question}")
        
        # Return prediction with context included for evaluation
        return dspy.Prediction(
            answer=answer.answer,
            context=context,  # Include context for the judge
            retrieved_ideas=relevant_ideas
        )

@dataclass
class PhilosophicalExample:
    """Example structure for philosophical Q&A evaluation."""
    question: str
    context: str
    expected_themes: List[str] = None

class IdeasConsistencyPipeline:
    """Fun pipeline for evaluating answers against personal philosophy."""
    
    def __init__(self, corpus_passages: List[str]):
        self.rag_program = MyIdeasRAGProgram(corpus_passages)
        self.corpus = corpus_passages
        
    def create_test_examples(self) -> List[PhilosophicalExample]:
        """Create interesting test questions based on your ideas."""
        test_questions = [
            "How should I deal with negative people in my life?",
            "What's the best way to express my true self?",
            "How can I find more beauty in everyday moments?",
            "What meditation techniques work best for daily awareness?",
            "How do I handle constantly rising expectations?",
            "What's the relationship between concentration and self-expression?",
            "How can I minimize the impact of petty judgments from others?",
            "What makes a moment truly beautiful?",
            "How do I balance self-discipline with authentic expression?",
            "What's the connection between awareness and time?"
        ]
        
        examples = []
        for question in test_questions:
            # Get relevant context for each question
            relevant_ideas = self.rag_program.retrieve_relevant_ideas(question)
            context = "\n\n".join(relevant_ideas)
            
            examples.append(PhilosophicalExample(
                question=question,
                context=context
            ))
        
        return examples
    
    def evaluate_answer(self, question: str, answer: str, context: str = None):
        """Evaluate a single answer against the ideas corpus."""
        if context is None:
            context = "\n\n".join(self.rag_program.retrieve_relevant_ideas(question))
        
        # Create example-like structure for evaluation
        example = type('Example', (), {
            'question': question,
            'context': context
        })()
        
        pred = type('Prediction', (), {
            'answer': answer
        })()
        
        result = idea_consistency_metric(example, pred)
        return result
    
    def run_full_evaluation(self, num_examples: int = 5):
        """Run a full evaluation pipeline with generated answers."""
        test_examples = self.create_test_examples()
        results = []
        
        print("🧠 Personal Philosophy Consistency Evaluation Pipeline")
        print("=" * 60)
        
        for i, example in enumerate(test_examples[:num_examples]):
            print(f"\n📝 Question {i+1}: {example.question}")
            print("-" * 40)
            
            # Generate answer using RAG
            pred = self.rag_program(example.question)
            
            print(f"🤖 Generated Answer: {pred.answer}")
            print(f"📚 Retrieved Context: {pred.context[:200]}...")
            
            # Evaluate consistency
            eval_result = self.evaluate_answer(
                example.question, 
                pred.answer, 
                pred.context
            )
            
            print(f"✅ Consistent: {eval_result['consistent']}")
            print(f"📊 Score: {eval_result['score']}")
            print(f"💭 Explanation: {eval_result['explanation']}")
            
            results.append({
                'question': example.question,
                'answer': pred.answer,
                'consistent': eval_result['consistent'],
                'score': eval_result['score'],
                'explanation': eval_result['explanation']
            })
            
            print("\n" + "="*60)
        
        # Summary statistics
        avg_score = sum(r['score'] for r in results) / len(results)
        consistent_count = sum(1 for r in results if r['consistent'])
        
        print(f"\n🎯 EVALUATION SUMMARY:")
        print(f"Average Consistency Score: {avg_score:.2f}")
        print(f"Consistent Answers: {consistent_count}/{len(results)}")
        print(f"Consistency Rate: {consistent_count/len(results)*100:.1f}%")
        
        return results
    
    def interactive_mode(self):
        """Interactive mode for testing custom questions."""
        print("🎮 Interactive Ideas Consistency Checker")
        print("Ask questions about life philosophy and get evaluated answers!")
        print("Type 'quit' to exit\n")
        
        while True:
            question = input("💭 Your question: ").strip()
            if question.lower() == 'quit':
                break
                
            # Generate answer
            pred = self.rag_program(question)
            print(f"\n🤖 Answer: {pred.answer}")
            
            # Evaluate
            result = self.evaluate_answer(question, pred.answer, pred.context)
            
            print(f"✅ Consistent with your ideas: {result['consistent']}")
            print(f"📊 Consistency score: {result['score']}")
            print(f"💭 Judge's reasoning: {result['explanation']}")
            print(f"📚 Context used: {pred.context[:150]}...")
            print("-" * 60)

# Example usage and demo
if __name__ == "__main__":
    # Initialize the pipeline
    pipeline = IdeasConsistencyPipeline(MY_CORPUS_PASSAGES)
    
    # Example 1: Test a specific answer
    print("🧪 Testing specific answer consistency:")
    test_question = "How should I deal with criticism from others?"
    test_answer = "Just ignore all criticism and focus on maximizing beauty in your life while expressing yourself fully."
    
    result = pipeline.evaluate_answer(test_question, test_answer)
    print(f"Question: {test_question}")
    print(f"Answer: {test_answer}")
    print(f"Consistent: {result['consistent']}")
    print(f"Explanation: {result['explanation']}")
    
    print("\n" + "="*60)
    
    # Example 2: Run full evaluation
    print("🚀 Running full evaluation pipeline:")
    pipeline.run_full_evaluation(num_examples=3)
    
    # Example 3: Interactive mode (commented out for demo)
    # pipeline.interactive_mode()

AttributeError: module 'dspy' has no attribute 'Signature'